# Notebook Test for Project

> A test for bpm, spo2 and respirat.

In [ ]:
# | default_exp core

In [ ]:
# | export
# the sample is from http://git.xinktech.com/yscz/alg/hr_detect_server/uploads/81dac5cbf6904113c61dee9305e9eada/sample.mp4
#video_path = "/home/wangli/projects/hr_detect_server/sample.mp4" 
video_path = video_DIR = '/home/wangli/data/pulse_rate/aliyun/2023.7.26/15.mp4' #fps is 25
server_ip = "localhost:8293"  # ip and port of the server.
hr_server_url = "http://{}/".format(server_ip)  # url of the server.

In [ ]:
# | hide
import base64
import time
import json
import os
import cv2
import requests

In [ ]:
# | export
def send_frame(frame, frame_id):
    image = cv2.imencode(".jpg", frame)[1]
    base64_data = str(base64.b64encode(image))[2:-1]
    datas = {"image": base64_data, "image_id": frame_id, "timestamp": int(time.time())}

    result = requests.post(
        hr_server_url + "process_framebyframe/", data=json.dumps(datas)
    )
    if result.status_code == 200:
        result = result.json()
        return result
    else:
        return None


# | export
def detect_pulse(device):
    records = {"bpm": [], "spo2": [], "respirat": []}

    if os.path.isdir(device):
        cap = cv2.VideoCapture(device + "//img_%04d.png")
    else:
        cap = cv2.VideoCapture(device)

    result = requests.post(
        hr_server_url + "start_hr_detect/?camera_fps=25&heart_interval=100&save=False"
    )
    print("start statue:", result)

    frame_num = 0
    while cap.isOpened():
        ret, frame = cap.read()

        if ret:  # 若是读取成功
            res = send_frame(frame, "{}".format(frame_num))
            print(res)
            frame_num += 1
        else:
            cap.release()
            break

        if frame_num % 50 == 0:
            res = requests.get(hr_server_url + "get_bpm/")
            if res.json() != {}:
                records["bpm"] += list(res.json().values())
                print(res.json())
            res = requests.get(hr_server_url + "get_respirat/")
            if res.json() != {}:
                records["respirat"] += list(res.json().values())
                print("respirat:", res.json())
            res = requests.get(hr_server_url + "get_spo2/")
            if res.json() != {}:
                records["spo2"] += list(res.json().values())
                print("spo2 level:", res.json())

    return records

In [ ]:
start_time = time.time()

results = detect_pulse(video_path)

end_time = time.time()
run_time = end_time - start_time
print("Function run time: ", run_time)

start statue: <Response [200]>
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
{'199': [0, [68.85], [1.53], True]}
spo2 level: {'199': [0, [94.34], [0], True]

In [ ]:
import plotly.graph_objects as go
import numpy as np


def get_signal_snr(records, name):
    sig = [r[1][0] for r in records[name]]
    snr = [r[2][0] for r in records[name]]

    return sig, snr


# | export
def plot_physics(records):
    names = ["bpm", "spo2", "respirat"]
    colors = ["red", "green", "blue"]
    trances = []
    for i in range(len(names)):
        name = names[i]
        sig, snr = get_signal_snr(records, name)
        if name == "respirat":
            start = 16
        else:
            start = 8
        x = np.arange(start=start, stop=start + len(sig))
        trace1 = go.Scatter(
            x=x, y=sig, mode="lines+markers", name=names[i], line=dict(color=colors[i])
        )
        trace2 = go.Scatter(
            x=x,
            y=snr,
            mode="lines+markers",
            name=names[i] + " snr",
            line=dict(dash="dash", color=colors[i]),
        )
        trances.append(trace1)
        trances.append(trace2)

    fig = go.Figure(data=trances)

    # 设置图表标题和轴标签
    fig.update_layout(title="Signal Curves", xaxis_title="X-axis", yaxis_title="Y-axis")

    # 显示图表
    fig.show()

In [ ]:
plot_physics(results)

In [ ]:
print(results)

{'bpm': [[0, [68.85], [1.53], True], [0, [66.65], [3.62], True], [0, [66.65], [3.62], True], [0, [66.65], [3.62], True], [0, [66.65], [3.62], True], [0, [68.85], [1.53], True], [0, [68.85], [1.53], True], [0, [68.85], [1.53], True], [0, [68.85], [1.53], True], [0, [73.24], [-0.81], True], [0, [72.51], [0.24], True], [0, [71.78], [-1.71], True]], 'spo2': [[0, [94.34], [0], True], [0, [94.34], [0], True], [0, [93.43], [0], True], [0, [93.56], [0], True], [0, [93.59], [0], True], [0, [93.59], [0], True], [0, [93.81], [0], True], [0, [94.22], [0], True], [0, [94.34], [0], True], [0, [94.7], [0], True], [0, [94.75], [0], True], [0, [94.76], [0], True]], 'respirat': [[0, [25.63], [-4.56], True], [0, [25.63], [-4.08], True], [0, [25.63], [-4.08], True], [0, [25.63], [-4.56], True], [0, [24.9], [-4.88], True], [0, [23.44], [-5.44], True], [0, [21.97], [-5.31], True], [0, [21.24], [-6.31], True]]}
